#### Import Required Libraries and Spark SQL Functions

In [0]:
import os
from pyspark.sql.functions import *

#### Define Gold and Silver Layer Base Paths

In [0]:
silver_base = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/silver"
gold_base   = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/gold"

#### Load and Prepare Silver Customer Data

In [0]:
df_dim_customer = (spark.read.format("delta")
                .load(f"{silver_base}/silver_adventureworks_customers")
                .withColumn("Age", floor(datediff(current_date(), col("BirthDate")) / 365))
                .withColumn(
                    "CustomerSegment",
                    when(col("AnnualIncome") >= 100000, "High Income")
                    .when(col("AnnualIncome") >= 50000, "Medium Income")
                    .otherwise("Low Income")
                )
                .select(
                    "CustomerKey",
                    "FullName",
                    "Gender",
                    "MaritalStatus",
                    "EducationLevel",
                    "Occupation",
                    col("AnnualIncome").cast("double"),
                    "Age",
                    "CustomerSegment"
                )
            )

In [0]:
df_dim_customer.limit(7).display()

CustomerKey,FullName,Gender,MaritalStatus,EducationLevel,Occupation,AnnualIncome,Age,CustomerSegment
11000,Mr. Jon Yang,M,M,Bachelors,Professional,90000.0,59,Medium Income
11001,Mr. Eugene Huang,M,S,Bachelors,Professional,60000.0,60,Medium Income
11002,Mr. Ruben Torres,M,M,Bachelors,Professional,60000.0,60,Medium Income
11003,Ms. Christy Zhu,F,S,Bachelors,Professional,70000.0,57,Medium Income
11004,Mrs. Elizabeth Johnson,F,S,Bachelors,Professional,80000.0,57,Medium Income
11005,Mr. Julio Ruiz,M,S,Bachelors,Professional,70000.0,60,Medium Income
11007,Mr. Marco Mehta,M,M,Bachelors,Professional,60000.0,61,Medium Income


#### Write Data to Gold Layer

In [0]:
df_dim_customer.write.format("delta")\
            .mode("overwrite")\
            .save(f"{gold_base}/dim_customer")
